In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op
from utils.batcher import generate_epoch

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [19]:
EMBEDDING_SIZE=32
HIDDEN_SIZE=128
BATCH_SIZE=16
NUM_STEPS=16
NUM_EPOCHS_INIT_LR=2
NUM_EPOCHS_TOTAL=10
INITIAL_LR=1.
LR_DECAY_RATE=0.5
MAX_NORM=1.

In [20]:
tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=vocab_size,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=vocab_size,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
train_op = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
    placeholders['max_norm'],
)

In [21]:
training_outputs = {**summary_nodes, 'train_op': train_op}
with tf.Session() as sess:

    # Bookkeeping
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    for i in range(NUM_EPOCHS_TOTAL):
        if i >= NUM_EPOCHS_INIT_LR:
            learning_rate *= LR_DECAY_RATE
        for batch_idx, (inputs, targets) in enumerate(generate_epoch(X_train, BATCH_SIZE, NUM_STEPS)):
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets,
                    placeholders['learning_rate']: learning_rate,
                    placeholders['max_norm']: max_norm,
                }
            )
            if (batch_idx % 256 == 255):
                print('step: {0}    loss: {1}    correct words: {2}'.format(
                    batch_idx+1,
                    outputs['loss'],
                    outputs['num_correct_predictions']
                ))
                
        total_loss, total_steps = 0, 0
        for inputs, targets in generate_epoch(X_train, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_steps += NUM_STEPS
        print('training perplexity:', np.exp(total_loss / (total_steps * BATCH_SIZE)) 
        total_loss, total_steps = 0, 0
        for inputs, targets in generate_epoch(X_val, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_steps += NUM_STEPS
        print('validation perplexity:', np.exp(total_loss / (total_steps * BATCH_SIZE))) 
        total_loss, total_steps = 0, 0
        for inputs, targets in generate_epoch(X_test, BATCH_SIZE, NUM_STEPS):
            outputs = sess.run(
                summary_nodes,
                feed_dict={
                    placeholders['inputs']: inputs,
                    placeholders['targets']: targets
                },
            )
            total_loss += outputs['loss']
            total_steps += NUM_STEPS
        print('test perplexity:', np.exp(total_loss / (total_steps * BATCH_SIZE))) 

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)

step: 256    loss: 103.3125    correct words: 22
step: 512    loss: 103.375    correct words: 21
step: 768    loss: 98.375    correct words: 28
step: 1024    loss: 100.6875    correct words: 37
step: 1280    loss: 99.75    correct words: 27
step: 1536    loss: 97.0625    correct words: 33
step: 1792    loss: 93.75    correct words: 42
step: 2048    loss: 88.375    correct words: 45
step: 2304    loss: 91.9375    correct words: 36
step: 2560    loss: 90.25    correct words: 34
step: 2816    loss: 96.0    correct words: 33
step: 3072    loss: 79.5    correct words: 57
step: 3328    loss: 86.5    correct words: 45
step: 3584    loss: 84.8125    correct words: 47
training perplexity: 228.179073756
validation perplexity: 243.239287593
test perplexity: 235.421043178
step: 256    loss: 88.125    correct words: 41
step: 512    loss: 89.375    correct words: 43
step: 768    loss: 86.125    correct words: 42
step: 1024    loss: 87.875    correct words: 54
step: 1280    loss: 89.8125    correct w